# Drug Portfolio Selection

Team: Adetoun, Chip, Lily, Matthias, Youssef

Due: 2021-12-02

## Setup

In [1]:
import gurobipy as gp
from gurobipy import GRB
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

## Data and Assumptions

*Author's note: we transformed the drug project data into a tidy format for ease of use. Some steps from the assignment template have been revised accordingly.*

Drug project data:

In [2]:
# Import drug data and transform into a tidy data frame
data = pd.read_csv('drugs.csv', index_col=0, header=None).transpose()
data = data.set_axis(['project', 'ta','time_to_market','revenue','cost'], axis=1, inplace=False)
data = data.astype({'project': int, 'ta': str, 'time_to_market': int, 'revenue': float, 'cost': float})

projects = data['project']
t_area = data['ta']           # therapeutic area
ttm = data['time_to_market']  # in years (whole numbers)
rev = data['revenue']         # in millions
cost = data['cost']           # in millions

#import covariance matrix
cov=pd.read_csv('drugs_cov.csv', index_col=0)

# creates lower triangular matrix needed in Value at Risk analysis in Q4 using Cholesky factorization
# Note that this results in L to be in matrix format (i.e., not in the dataframe format anymore)
L = np.linalg.cholesky(cov)

Therapeutic areas, budgets, and risk-free rate of return:

In [3]:
#therapeutic areas
ther=[
"Oncology", "Cardiovascular", "Respiratory and dermatology", "Transplantation",
"Rheumatology and hormone therapy", "Central nervous system", "Ophtalmics"]

#budget constraints for therapeutic areas
t_bud={"Oncology": 100,
       "Cardiovascular": 200,
       "Respiratory and dermatology": 150,
       "Transplantation": 100,
       "Rheumatology and hormone therapy": 300,
       "Central nervous system": 100,
       "Ophtalmics": 50}

interest_rate=0.03

base_budget=1000

additional_budget=50

## Model 2A - Without $50MM Extra

Initialize empty model:

In [4]:
m = gp.Model('portfolio')

Academic license - for non-commercial use only - expires 2022-09-25
Using license file /Users/youssefragab/gurobi.lic


Decision variable (whether or not to invest in each project):

In [5]:
x = pd.Series(m.addVars(projects, vtype = GRB.BINARY), index=projects)

In [6]:
#define covariance for each project
portfolio_risk = np.dot(np.dot(np.transpose(x),cov), x)

Constraints:

In [7]:
# $1 billion total budget
m.addConstr(sum(x[i] * cost[i] for i in projects) <= base_budget)

# TA-level budgets
for t in ther:
  m.addConstr(sum(x[i] * cost[i] for i in projects if t_area[i] == t) <= t_bud[t])

# Pipeline balance
m.addConstr(sum(x[i] for i in projects if ttm[i] == 1) >= 0.15 * sum(x[i] for i in projects))
m.addConstr(sum(x[i] for i in projects if ttm[i] in (2,3)) >= 0.20 * sum(x[i] for i in projects))
m.addConstr(sum(x[i] for i in projects if ttm[i] in (4,5)) >= 0.25 * sum(x[i] for i in projects))

#risk constraint which minimizes the variance of the project returns
#comment constraint out and rerun model at different variance levels to capture return
m.addConstr(portfolio_risk<=1.59E+04)

# For validation
m.write("model.lp")


Objective function (maximize revenue plus return on uninvested funds minus total budget):

In [8]:
tot_rev = sum(x[i] * rev[i] for i in projects)

tot_cost = sum(x[i] * cost[i] for i in projects)

uninvested_return = interest_rate * (base_budget - tot_cost)

m.setObjective(tot_rev + uninvested_return - base_budget, GRB.MAXIMIZE)
  
# For validation
m.write("model.lp")

### Model 2A Results

In [9]:
# Optimize model to find max rev
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 11 rows, 114 columns and 568 nonzeros
Model fingerprint: 0x39450584
Model has 1 quadratic constraint
Variable types: 0 continuous, 114 integer (114 binary)
Coefficient statistics:
  Matrix range     [1e-01, 2e+02]
  QMatrix range    [1e-01, 5e+06]
  Objective range  [1e+00, 3e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+03]
  QRHS range       [2e+04, 2e+04]
Found heuristic solution: objective -970.0000000
Presolve removed 8 rows and 39 columns
Presolve time: 0.00s
Presolved: 3 rows, 75 columns, 225 nonzeros
Presolved model has 1 quadratic constraint(s)
Variable types: 0 continuous, 75 integer (75 binary)

Root relaxation: objective 3.646727e+02, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It

In [10]:
print('Variance   = %g' % portfolio_risk.getValue())

Variance   = 14330.1


In [11]:
# Create an expression representing the expected return for the portfolio
portfolio_return = tot_rev

In [12]:
# total return = revenue plus return on uninvested funds minus budget
print("total return: ", m.objVal) 

total return:  -31.648900000000026


In [13]:
for v in m.getVars():
  if v.x == 1:
    print("Invested in project", v.varName)

Invested in project C44
Invested in project C58
Invested in project C68
Invested in project C72
Invested in project C97
Invested in project C110


In [14]:
# this loop feels a really dumb way to calculate this, feel free to improve
spent = 0
n = 0
for i in projects:
  if m.getVars()[i - 1].x == 1:
    spent += cost[i]
    n += 1

print("Invested", spent, "million dollars into", n, "projects.")

Invested 65.63 million dollars into 6 projects.


### Model 2A Efficient Frontier Illustration

Click to view [Efficient Frontier illustration](https://drive.google.com/file/d/1JiSkXC6R83v1NpH0Z7l0lOs70cRGVP9N/view?usp=sharing)

## Model 2B - With $50MM Extra

Run model with different RHS values for the right hand side of the portfolio risk constraint:

In [15]:
portfolio_risk_rhs = 1.80E+07

In [16]:
# Initialize similar model
m = gp.Model('portfolio')

# With similar decision variables
x = m.addVars(projects, vtype = GRB.BINARY, name = 'x')

Additional budget helpers:

In [17]:
# Decision variable for assigning additional budget to TAs
b = m.addVars(ther, vtype = GRB.BINARY, name = 'b')

# Dictionary of additional funds per TA
t_bud_extra = t_bud.copy()

# Assign zero or additional budget to the copy dictionary
for t in ther:
  t_bud_extra[t] = additional_budget * b[t]

# Allow up to 1 additional budget assignment
m.addConstr(sum(b[t] for t in ther) <= 1)

# Note: the TA-level budget constraint in the next cell has also been updated

<gurobi.Constr *Awaiting Model Update*>

Similar constraints and objective function as before (except for the additional budget):

In [18]:
# $1 billion total budget
m.addConstr(sum(x[i] * cost[i] for i in projects) <= base_budget + additional_budget)

# TA-level budgets (base plus extra $50MM)
for t in ther:
  m.addConstr(sum(x[i] * cost[i] for i in projects if t_area[i] == t) <= t_bud[t] + t_bud_extra[t])

# Pipeline balance
m.addConstr(sum(x[i] for i in projects if ttm[i] == 1) >= 0.15 * sum(x[i] for i in projects))
m.addConstr(sum(x[i] for i in projects if ttm[i] in (2,3)) >= 0.20 * sum(x[i] for i in projects))
m.addConstr(sum(x[i] for i in projects if ttm[i] in (4,5)) >= 0.25 * sum(x[i] for i in projects))

# Objective function
tot_rev = sum(x[i] * rev[i] for i in projects)
tot_cost = sum(x[i] * cost[i] for i in projects)
uninvested_return = interest_rate * (base_budget - tot_cost)
m.setObjective(tot_rev + uninvested_return - base_budget, GRB.MAXIMIZE)
  
# For validation
m.write("model.lp")

Add a portfolio risk constraint.  Use a subset of the constraints used to make the efficient frontier.

In [19]:
#define covariance for each project
x_for_risk = pd.Series(x, index=projects)
portfolio_risk = np.dot(np.dot(np.transpose(x_for_risk),cov), x_for_risk)

In [20]:
m.addConstr(portfolio_risk <= portfolio_risk_rhs)

<gurobi.QConstr Not Yet Added>

### Model 2B Results

In [21]:
m.setParam('OutputFlag', 0) # run silently
m.optimize()

In [22]:
print("Total return: ", m.objVal) 

print('Variance   = %g' % portfolio_risk.getValue())

Total return:  21769.355499999998
Variance   = 1.79976e+07


In [23]:
for v in m.getVars():
  if v.x == 1:
    print("Invested in project", v.varName)

Invested in project x[3]
Invested in project x[4]
Invested in project x[5]
Invested in project x[6]
Invested in project x[17]
Invested in project x[20]
Invested in project x[21]
Invested in project x[22]
Invested in project x[24]
Invested in project x[25]
Invested in project x[26]
Invested in project x[27]
Invested in project x[28]
Invested in project x[29]
Invested in project x[30]
Invested in project x[39]
Invested in project x[40]
Invested in project x[42]
Invested in project x[44]
Invested in project x[45]
Invested in project x[46]
Invested in project x[47]
Invested in project x[48]
Invested in project x[50]
Invested in project x[53]
Invested in project x[57]
Invested in project x[58]
Invested in project x[61]
Invested in project x[62]
Invested in project x[66]
Invested in project x[69]
Invested in project x[72]
Invested in project x[76]
Invested in project x[77]
Invested in project x[78]
Invested in project x[86]
Invested in project x[87]
Invested in project x[91]
Invested in proj

### Model 2B Remarks

The optimal therapeutic area deployment for the extra $50MM depends on Zinca's risk tolerance. With high tolerance, the company should invest in Respiratory and Dermatology because that choice maximizes total return (over $20Bn).  With the least tolerance, they should invest in Central nervous system (although this strategy barely breaks even, e.g. returning only $16K on a portfolio with 126 standard deviation).

Without knowing Zinca's risk tolerance, I would recommend Respiratory and Dermatology.  This is because there is a clear elbow in the efficiency frontier, and this therapeutic area straddles both sides of that elbow.  This is a good sign that choosing that TA offers an efficient risk:reward ratio.